In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Step 1: Device 설정 (GPU 사용 가능 여부 확인)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Step 2: STL-10 데이터셋 로드 및 전처리 설정
def load_dataset():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet이 기대하는 입력 크기
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 일반적인 ImageNet 정규화 값
    ])

    train_dataset = datasets.STL10(root='.', split='train', download=True, transform=transform)
    test_dataset = datasets.STL10(root='.', split='test', download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    classes = train_dataset.classes
    return train_loader, test_loader, classes

In [3]:
# Step 3: ImageNet 사전 학습된 ResNet 모델 불러오기 및 Fine-Tuning 설정
model = models.resnet18(pretrained=True)  # ImageNet으로 사전 학습된 ResNet18 사용
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # STL-10 클래스 수에 맞게 출력층 수정
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# 데이터셋 로드
train_loader, test_loader, classes = load_dataset()

# Step 4: Fine-Tuning 수행
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # 학습 모드로 설정
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # GPU로 이동
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Loss: 0.6552
Epoch [2/5], Loss: 0.1703
Epoch [3/5], Loss: 0.1036
Epoch [4/5], Loss: 0.0599
Epoch [5/5], Loss: 0.0386


In [4]:
# Fine-Tuning 완료 후 모델 가중치 저장
torch.save(model.state_dict(), "STL10-finetuned_model.pth")

In [5]:
# Step 5: 모델 평가
model.eval()  # 평가 모드로 설정
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # GPU로 이동
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 95.67%


In [6]:
# Step 6: 예측 함수 - 새로운 이미지에 대한 예측 수행
def predict(image):
    model.eval()
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = preprocess(image).unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

    with torch.no_grad():
        output = model(input_tensor).squeeze(0).softmax(0)  # 확률값으로 구성된 1차원 배열
    class_id = output.argmax().item()  # 예측된 클래스 ID
    score = output[class_id].item()  # 해당 클래스의 확률값
    class_name = classes[class_id]
    print(f"Predicted: {class_name} ({100 * score:.1f}%)")

    # 1차원 확률 배열 반환
    return output

In [7]:
# 예시
from PIL import Image

# 로컬 이미지 파일 경로
image_path = "./Car1.jpg"
image = Image.open(image_path)

# 예측 수행 및 확률값 출력
probabilities = predict(image)
print("Class Probabilities:", probabilities)

Predicted: car (100.0%)
Class Probabilities: tensor([1.4889e-04, 4.4769e-05, 9.9955e-01, 1.7002e-05, 1.8168e-05, 8.5754e-05,
        2.4666e-05, 2.4718e-05, 7.7311e-06, 7.9653e-05], device='cuda:0')


### Serving Try 1

In [8]:
!pip install fastapi uvicorn torch torchvision pillow

In [9]:
!pip install python-multipart

In [10]:
import torch
from torchvision import transforms, models
from PIL import Image
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from io import BytesIO

# 모델 및 전처리 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사전 학습된 모델 로드 및 Fine-Tuning 된 분류층 불러오기
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 10)  # STL-10에 맞게 클래스 수를 10으로 설정
model.load_state_dict(torch.load("STL10-finetuned_model.pth", map_location=device))  # Fine-Tuning된 모델 가중치 로드
model = model.to(device)
model.eval()

# 전처리 파이프라인 설정
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 클래스 이름 (STL-10 클래스)
classes = ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]

# FastAPI 인스턴스 생성
app = FastAPI()

# Request 형식 정의
class PredictionResponse(BaseModel):
    class_name: str
    confidence: float

# 이미지 예측 함수
def predict(image: Image.Image):
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor).squeeze(0).softmax(0)
    class_id = output.argmax().item()
    confidence = output[class_id].item()
    return classes[class_id], confidence

# API 엔드포인트 정의
@app.post("/predict", response_model=PredictionResponse)
async def predict_image(file: UploadFile = File(...)):
    # 이미지 파일 열기
    image = Image.open(BytesIO(await file.read())).convert("RGB")
    # 예측 수행
    class_name, confidence = predict(image)
    # 결과 반환
    return {"class_name": class_name, "confidence": confidence}

<ipython-input-10-ab808c648f94>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("STL10-finetuned_model.pth", map_location=device))  # Fine

In [11]:
# app.py 파일에 Serving 코드 저장하기
serving_code = '''
import torch
from torchvision import transforms, models
from PIL import Image
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from io import BytesIO

# 모델 및 전처리 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사전 학습된 모델 로드 및 Fine-Tuning 된 분류층 불러오기
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 10)  # STL-10에 맞게 클래스 수를 10으로 설정
model.load_state_dict(torch.load("STL10-finetuned_model.pth", map_location=device))  # Fine-Tuning된 모델 가중치 로드
model = model.to(device)
model.eval()

# 전처리 파이프라인 설정
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 클래스 이름 (STL-10 클래스)
classes = ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]

# FastAPI 인스턴스 생성
app = FastAPI()

# Request 형식 정의
class PredictionResponse(BaseModel):
    class_name: str
    confidence: float

# 이미지 예측 함수
def predict(image: Image.Image):
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor).squeeze(0).softmax(0)
    class_id = output.argmax().item()
    confidence = output[class_id].item()
    return classes[class_id], confidence

# API 엔드포인트 정의
@app.post("/predict", response_model=PredictionResponse)
async def predict_image(file: UploadFile = File(...)):
    # 이미지 파일 열기
    image = Image.open(BytesIO(await file.read())).convert("RGB")
    # 예측 수행
    class_name, confidence = predict(image)
    # 결과 반환
    return {"class_name": class_name, "confidence": confidence}
'''

# 파일에 코드 저장
with open("STL10-serving.py", "w") as f:
    f.write(serving_code)

### Serving: ngrok
--> Well Executed

In [12]:
!pip install pyngrok

In [13]:
from pyngrok import ngrok

# ngrok 인증 토큰 설정
ngrok.set_auth_token("2oBL7F32G2iIEV5ie5dAumgGMXc_2ZLm9JGC1ZPnNbgwDG6P3")

In [14]:
from pyngrok import ngrok

# 실행 중인 모든 ngrok 터널 종료
ngrok.kill()

In [15]:
from pyngrok import ngrok
import uvicorn
import threading

# ngrok을 사용하여 포트 8002을 공개합니다.
public_url = ngrok.connect(8002, "http")
print("ngrok Public URL:", public_url)

# FastAPI 서버를 백그라운드에서 실행하는 함수
def run_app():
    uvicorn.run("STL10-serving:app", host="0.0.0.0", port=8002)

# 별도의 스레드에서 FastAPI 서버 실행
thread = threading.Thread(target=run_app)
thread.start()

ngrok Public URL: NgrokTunnel: "https://b4a1-34-118-243-130.ngrok-free.app" -> "http://localhost:8002"


In [16]:
import os

file_path = "/content/Car1.jpg"
print("File exists:", os.path.exists(file_path))

File exists: True
